In [127]:
import pandas as pd
from datetime import date

df_players = pd.read_csv(r"bases_originais\Base_Cadastral_de_Players.csv", sep =";")
df_transacoes = pd.read_csv(r"bases_originais\Base_de_Transacoes_e_Cupons_Capturados.csv", sep =";")
df_lojas = pd.read_csv(r"bases_originais\Base_Massa_de_Teste_com_Lojas_e_Valores.csv", sep =";")
df_simulacao = pd.read_csv(r"bases_originais\Base_Simulada_-_Pedestres_Av__Paulista.csv", sep =";")

# Tratamentos Base de Players
- ajuste do formato da coluna data_nascimento para aaaa-mm-dd
- garantir que a idade bata com a data de nascimento

In [128]:
df = df_players

df["data_nascimento"] = pd.to_datetime(df["data_nascimento"], dayfirst=True, errors="coerce")

hoje = pd.Timestamp(date.today())

df["idade"] = df["data_nascimento"].apply(
    lambda d: hoje.year - d.year - ((hoje.month, hoje.day) < (d.month, d.day)) if pd.notnull(d) else None
)



In [129]:
df.to_csv(r"bases_tratadas\base_players.csv", index=False, encoding="utf-8-sig")

# Tratamentos Base de Transações
- ajuste do formato da coluna data para aaaa-mm-dd
- ajuste do forma da coluna hora para o formato hh:mm:ss
- ajustes das categorias dos estabelecimentos
- limpar os dados da coluna produtos
- inserir o campo nome_campanha replicando os dados de id_campanha para fins de teste

In [130]:
df = df_transacoes

df["data"] = pd.to_datetime(df["data"], dayfirst=True, errors="coerce")
df["hora"] = pd.to_datetime(df["hora"], errors="coerce").dt.time


C:\Users\dudal\AppData\Local\Temp\ipykernel_9856\959563625.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["hora"] = pd.to_datetime(df["hora"], errors="coerce").dt.time


In [131]:
estabelecimentos = df["nome_estabelecimento"].unique()

for estabelecimento in estabelecimentos:
    print(f"'{estabelecimento}':'")

'Habib's':'
'Smart Fit':'
'Outback':'
'Subway':'
'Octavio Café':'
'Madero':'
'Forever 21':'
'Sabin':'
'Lavoisier':'
'Renner':'
'Starbucks':'
'Ponto':'
'Café Cultura':'
'Selfit':'
'Droga Raia':'
'Extra':'
'Just Run':'
'Clube Pinheiros':'
'Carrefour Express':'
'Sesc Paulista':'
'Drogasil':'
'Drogaria São Paulo':'
'Riachuelo':'
'Burger King':'
'Sesc Carmo':'
'Casas Bahia':'
'Açaí no Ponto':'
'Churrascaria Boi Preto':'
'Fleury':'
'Magazine Luiza':'
'McDonald's':'
'Pão de Açúcar':'
'Ráscal':'


In [132]:
from math import nan
from tokenize import group


mapa_categorias = {
    "Habib's": "Restaurante",
    "Burger King": "Fast Food",
    "Outback": "Restaurante",
    "Subway": "Fast Food",
    "Madero": "Restaurante",
    "Ráscal": "Restaurante",
    "Churrascaria Boi Preto": "Restaurante",
    "Açaí no Ponto": "Lanchonete",
    "Starbucks": "Cafeteria",
    "Café Cultura": "Cafeteria",
    "Octavio Café": "Cafeteria",
    "Smart Fit": "Academia",
    "Selfit": "Academia",
    "Just Run": "Academia",
    "Clube Pinheiros": "Clube",
    "Sesc Paulista": "Centro Esportivo",
    "Sesc Carmo": "Centro Esportivo",
    "Forever 21": "Vestuário",
    "Renner": "Vestuário",
    "Riachuelo": "Vestuário",
    "Magazine Luiza": "Varejo",
    "Casas Bahia": "Varejo",
    "Ponto": "Varejo",
    "Extra": "Supermercado",
    "Pão de Açúcar": "Supermercado",
    "Carrefour Express": "Supermercado",
    "Droga Raia": "Farmácia",
    "Drogasil": "Farmácia",
    "Drogaria São Paulo": "Farmácia",
    "Sabin": "Laboratório",
    "Fleury": "Laboratório",
    "Lavoisier": "Laboratório",
    "McDonald's": "Fast Food"
}

df["categoria_estabelecimento"] = df["nome_estabelecimento"].map(mapa_categorias)

In [133]:
df.insert(7, "nome_campanha", df["id_campanha"])

In [134]:
df.to_csv(r"bases_tratadas\base_transacoes.csv", index=False, encoding="utf-8-sig")


# Tratamento Base de Lojas
- ajuste do formato da coluna data_captura para aaaa-mm-dd
- ajustes os tipo_loja pelo nome_loja
- alterar o nome das colunas numero_celular, tipo_loja, nome_loja, data_captura, endereco_loja


In [135]:
df = df_lojas

df["data_captura"] = pd.to_datetime(df["data_captura"], dayfirst=True, errors="coerce")

In [136]:
#listar lojas atuais
lojas = df["nome_loja"].unique()

for loja in lojas:
    print(f"'{loja}':'',")

'Pão de Açúcar':'',
'Kalunga':'',
'Daiso Japan':'',
'Smart Fit':'',
'Livraria Cultura':'',
'Drogaria São Paulo':'',
'Havaianas':'',
'Subway':'',
'Lojas Americanas':'',
'Fast Shop':'',
'Outback':'',
'Extra Mercado':'',
'Ponto Frio':'',
'Renner':'',
'Riachuelo':'',


In [137]:
tipos = {
'Pão de Açúcar':'Super-Mercado',
'Kalunga':'Papelaria',
'Daiso Japan':'Eletrônicos',
'Smart Fit':'Acadêmia',
'Livraria Cultura':'Livraria',
'Drogaria São Paulo':'Farmácia',
'Havaianas':'Vestuário',
'Subway':'Fast Food',
'Lojas Americanas':'Super-Mercado',
'Fast Shop':'Eletrônicos',
'Outback':'Restaurante',
'Extra Mercado':'Super-Mercado',
'Ponto Frio':'Varejista',
'Renner':'Vestuário',
'Riachuelo':'Vestuário'
}

df["tipo_loja"] = df["nome_loja"].map(mapa_categorias)

In [138]:
df = df.rename(columns={
    "numero_celular": "celular",
    "data_captura" : "data",
    "tipo_loja" : "categoria_estabelecimento", 
    "nome_loja" : "nome_estabelecimento",
    "endereco_loja" : "endereco_estabelecimento"
    })


In [139]:
df.to_csv(r"bases_tratadas\base_lojas.csv", index=False, encoding="utf-8-sig")

# Tratamentos Base de Simulação
- ajuste do formato da coluna data e data_ultima_compra para aaaa-mm-dd
- ajuste do forma da coluna horario para o formato hh:mm:ss

In [140]:
df = df_simulacao

df["data"] = pd.to_datetime(df["data"], dayfirst=True, errors="coerce")
df["data_ultima_compra"] = pd.to_datetime(df["data_ultima_compra"], dayfirst=True, errors="coerce")
df["horario"] = pd.to_datetime(df["horario"], errors="coerce").dt.time

C:\Users\dudal\AppData\Local\Temp\ipykernel_9856\4129307373.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["horario"] = pd.to_datetime(df["horario"], errors="coerce").dt.time


In [141]:
df.to_csv(r"bases_tratadas\base_simulacao.csv", index=False, encoding="utf-8-sig")